# M2608.001300 Machine Learning<br> Assignment #5 Final Projects (Pytorch)


Copyright (C) Data Science & AI Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them.

**For understanding of this work, please carefully look at given PPT file.**

Note: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.

In [8]:
from google.colab import drive
drive.mount('/content/drive')

import os
import random

import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
from PIL import Image
from PIL import ImageFilter
!pip3 install resnet
#import resnet
import torchvision.models as models
#!ls '/content/drive/My Drive/Data'
import cv2



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load datasets


In [9]:
NUMBER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
ALPHABET = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
NONE = ['NONE'] # label for empty space
ALL_CHAR_SET = NUMBER + ALPHABET + NONE
ALL_CHAR_SET_LEN = len(ALL_CHAR_SET)
MAX_CAPTCHA = 7

print(ALL_CHAR_SET.index('NONE'))

def encode(a):
    onehot = [0]*ALL_CHAR_SET_LEN
    idx = ALL_CHAR_SET.index(a)
    onehot[idx] += 1
    return onehot

# modified dataset class
class Mydataset(Dataset):
    def __init__(self, img_path, label_path, is_train=True, transform=None):
        self.path = img_path
        self.label_path = label_path
        if is_train: 
            self.img = os.listdir(self.path)[:10000]
            self.labels = open(self.label_path, 'r').read().split('\n')[:-1][:10000]
        else: 
            self.img = os.listdir(self.path)[:1000]
            self.labels = open(self.label_path, 'r').read().split('\n')[:-1][:1000]
        self.img=sorted(self.img, key=lambda a:(len(a),int(a[:-4])))
        self.transform = transform
        self.max_length = MAX_CAPTCHA
        
    def __getitem__(self, idx):
        img_path = self.img[idx]
        img = Image.open(f'{self.path}/{self.img[idx]}')
        img = img.convert('L')
        img=img.filter(ImageFilter.MinFilter(size=3))
        label = self.labels[idx]
        label_oh = []
        # one-hot for each character
        for i in range(self.max_length):
            if i < len(label):
                label_oh += encode(label[i])
            else:
                #label_oh += [0]*ALL_CHAR_SET_LEN
                label_oh += encode('NONE')
            
        if self.transform is not None:
            img = self.transform(img)
        return img, np.array(label_oh), label
    
    def __len__(self):
        return len(self.img)

transform = transforms.Compose([
    transforms.Resize([160, 60]),
    transforms.ToTensor(),
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################
    #preprocessed above.
##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################
])



36


In [10]:
"""Loading DATA"""
# Change to your own data folder path!
gPath = '/content/drive/My Drive/'

train_ds = Mydataset(gPath+'Data/train/', gPath+'Data/train.txt',transform=transform)
test_ds = Mydataset(gPath+'Data/test/', gPath+'Data/test.txt',False, transform)
#print(train_ds.__len__())
train_dl = DataLoader(train_ds, batch_size=8, num_workers=4) #128 no.
test_dl = DataLoader(test_ds, batch_size=1, num_workers=4)

#for i,j in enumerate(train_dl):
#  arr_=np.squeeze(j[0][1])
#  plt.imshow(arr_)
#  plt.show()

In [11]:
"""To CUDA for local run"""
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#GPUID = '4' # define GPUID
#os.environ["CUDA_VISIBLE_DEVICES"] = str(GPUID)

In [ ]:
I=6
vocab=37
hidden=8
batch=8
rnn = nn.LSTMCell(vocab, hidden)
input = torch.randn(I, batch, vocab)
hx = torch.randn(batch, hidden)
cx = torch.randn(batch, hidden)
output = []
for i in range(I):
        hx, cx = rnn(input[i], (hx, cx))
        output.append(hx)
        #print(hx.size())
#print(output)

Problem 1: Design LSTM model for catcha image recognition. (10 points)

In [13]:
cnn_dim=259 #resnet18-512
hidden_size=8

class LSTM(nn.Module):
    def __init__(self, cnn_dim, hidden_size, vocab_size, num_layers=1):
        super(LSTM, self).__init__()
        
        # define the properties
        self.cnn_dim = cnn_dim
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        
        # lstm cell
        self.lstm_cell = nn.LSTMCell(input_size=self.vocab_size, hidden_size=hidden_size)
    
        # output fully connected layer
        self.fc_in = nn.Linear(in_features=self.cnn_dim, out_features=self.vocab_size)
        self.fc_out = nn.Linear(in_features=self.hidden_size, out_features=self.vocab_size)
    
        # embedding layer
        self.embed = nn.Embedding(num_embeddings=self.vocab_size, embedding_dim=self.vocab_size)
    
        # activations
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, features, captions): 

        batch_size = features.size(0)
        cnn_dim = features.size(1)

        hidden_state = torch.zeros((batch_size, self.hidden_size)).cuda()
        cell_state = torch.zeros((batch_size, self.hidden_size)).cuda()
    
        # define the output tensor placeholder
        outputs = torch.empty((batch_size, captions.size(1), self.vocab_size)).cuda()

        # embed the captions
        captions_embed = self.embed(captions)

##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################
        for i in range(features.size()):
            hidden_state, cell_state = rnn(input[i], (hidden_state, cell_state))
            outputs[i]=hidden_state
##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################
        return outputs #8 7 37

lstm = LSTM(cnn_dim=cnn_dim, hidden_size=hidden_size, vocab_size=vocab_size)
#for i,j in enumerate(train_dl):
  #print(j[1][0])
  #print(lstm.forward(j[0],j[1]).size()) #16 7 37


Problem 2: 

*   1.Connect CNN model to the designed LSTM model.
*   2.Replace ResNet to your own CNN model from Assignment3.


          


In [12]:

##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################
"""ResNet"""


class Inception(nn.Module):
    def __init__(self, in_planes, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_planes):
        super(Inception, self).__init__()
        # 1x1 conv branch
        self.b1 = nn.Sequential(
            nn.Conv2d(in_planes, n1x1, kernel_size=1),
            nn.BatchNorm2d(n1x1),
            nn.ReLU(True),
        )

        # 1x1 conv -> 3x3 conv branch
        self.b2 = nn.Sequential(
            nn.Conv2d(in_planes, n3x3red, kernel_size=1),
            nn.BatchNorm2d(n3x3red),
            nn.ReLU(True),
            nn.Conv2d(n3x3red, n3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(n3x3),
            nn.ReLU(True),
        )

        # 1x1 conv -> 5x5 conv branch
        self.b3 = nn.Sequential(
            nn.Conv2d(in_planes, n5x5red, kernel_size=1),
            nn.BatchNorm2d(n5x5red),
            nn.ReLU(True),
            nn.Conv2d(n5x5red, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
            nn.Conv2d(n5x5, n5x5, kernel_size=3, padding=1),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
        )

        # 3x3 pool -> 1x1 conv branch
        self.b4 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_planes, pool_planes, kernel_size=1),
            nn.BatchNorm2d(pool_planes),
            nn.ReLU(True),
        )

    def forward(self, x):
        y1 = self.b1(x)
        y2 = self.b2(x)
        y3 = self.b3(x)
        y4 = self.b4(x)
        return torch.cat([y1,y2,y3,y4], 1)
class BetterNet(nn.Module):
    def __init__(self):
        super(BetterNet, self).__init__()
        self.preprocess = nn.Sequential(
          nn.Conv2d(1, 192, kernel_size=3, padding=1),
          nn.BatchNorm2d(192),
          nn.ReLU(True),
        )
        self.maxpool = nn.MaxPool2d(3, stride=2, padding=1)
        self.a_1 = Inception(192,  64,  96, 128, 16, 32, 32)
        self.b_1 = Inception(256, 128, 128, 192, 32, 96, 64)
        self.a_2 = Inception(480, 192,  96, 208, 16,  48,  64)
        self.b_2 = Inception(512, 160, 112, 224, 24,  64,  64)
        self.c_2 = Inception(512, 128, 128, 256, 24,  64,  64)
        self.d_2 = Inception(512, 112, 144, 288, 32,  64,  64)
        self.e_2 = Inception(528, 256, 160, 320, 32, 128, 128)
        self.a_3 = Inception(832, 256, 160, 320, 32, 128, 128)
        self.b_3 = Inception(832, 384, 192, 384, 48, 128, 128)
        self.avgpool = nn.AvgPool2d(8, stride=1)
        self.linear = nn.Linear(270336, 259*8) #259

    def forward(self, x):
        out = self.preprocess(x) # 8 192 160 60
        out = self.a_1(out)
        out = self.b_1(out)
        out = self.maxpool(out)
        out = self.a_2(out)
        out = self.b_2(out)
        out = self.c_2(out)
        out = self.d_2(out)
        out = self.e_2(out)
        out = self.maxpool(out)
        out = self.a_3(out)
        out = self.b_3(out)
        out = self.avgpool(out) # 8 1024 33 8
        out = out.view(out.size(0), -1)  #8  270336
        out = self.linear(out)
        out = out.view(out.size(0), 8,  -1)
        return out
#CNN
#betternet = models.resnet18(pretrained=False)
betternet=BetterNet()
#betternet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#betternet.fc = nn.Linear(in_features=512, out_features=ALL_CHAR_SET_LEN*MAX_CAPTCHA, bias=True)
betternet = betternet.to(device)
##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################

      
# LSTM
cnn_dim=259 #resnet18-512
hidden_size=8
vocab_size=37 #ALL_CHAR_SET_LEN
lstm1 = nn.LSTM(cnn_dim,hidden_size,vocab_size)#LSTM(cnn_dim=cnn_dim, hidden_size=hidden_size, vocab_size=vocab_size)
lstm1 = lstm1.to(device)
# loss, optimizer
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################
loss_func = nn.MultiLabelSoftMarginLoss()
cnn_optim = torch.optim.Adam(betternet.parameters(), lr=0.001)

##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################

In [14]:
PATH = './better_net.pth'

#with torch.no_grad():

optimizer = optim.SGD(betternet.parameters(), lr=0.001, momentum=0.9) #is 0.001 too high?
def train(net1, net2, trainloader, max_epoch, crit, opt, model_path='./cifar_net.pth'):

    for epoch in range(max_epoch):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs  =  data[0]
            labels = data[1]
            #for i in range(0,8):
            #  arr_=np.squeeze(inputs[i])
            #  plt.imshow(arr_)  
            #  plt.show()
            #print(labels)
            # Training on GPU
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs1 = net1(inputs)
            print(outputs1.size())
            loss1 = crit(outputs1, labels)
            loss1.backward()
            opt.step()

            outputs2 = net2(outputs1)
            loss2 = crit(outputs2, labels)
            loss2.backward()
            opt.step()

            # print statistics
            running_loss += loss1.item()+loss2.item()
            if i % 100 == 99:    # print every 300 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

    print('Finished Training')
    torch.save(net1.state_dict(), model_path)
    torch.save(net2.state_dict(), model_path)
    print('Saved Trained Model')
train(betternet,lstm1, train_dl, 5, loss_func, cnn_optim, PATH)

RuntimeError: ignored

Problem3: Find hyper-parameters.


In [ ]:
"""TRAINING"""


print_interval = 10
max_epoch = 10 #40,1000
#print(train_dl.__len__())
for epoch in range(max_epoch):
    for step, i in enumerate(train_dl):
        img, label_oh, label = i
        img = Variable(img).cuda()
        label_oh = Variable(label_oh.float()).cuda()
        batch_size, _ = label_oh.shape #4*259
        #pred, feature = betternet(img)
        pred = betternet(img)
        loss = loss_func(pred, label_oh)
        cnn_optim.zero_grad()
        loss.backward()
        cnn_optim.step()###should be changed
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################

##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################
        if (step+1)%print_interval == 0:
            print('epoch:', epoch+1, 'step:', step+1, 'loss:', loss.item())

epoch: 1 step: 10 loss: 0.08188681304454803
epoch: 1 step: 20 loss: 0.07848311960697174
epoch: 1 step: 30 loss: 0.08017425239086151
epoch: 1 step: 40 loss: 0.07288508117198944
epoch: 1 step: 50 loss: 0.08147762715816498
epoch: 1 step: 60 loss: 0.0734580010175705
epoch: 1 step: 70 loss: 0.06640581786632538
epoch: 1 step: 80 loss: 0.06938903778791428
epoch: 1 step: 90 loss: 0.07761746644973755
epoch: 1 step: 100 loss: 0.07943476736545563
epoch: 1 step: 110 loss: 0.06714969873428345
epoch: 1 step: 120 loss: 0.08524724841117859
epoch: 1 step: 130 loss: 0.07974059134721756
epoch: 1 step: 140 loss: 0.08511141687631607
epoch: 1 step: 150 loss: 0.07282016426324844
epoch: 1 step: 160 loss: 0.06584198027849197
epoch: 1 step: 170 loss: 0.06360799074172974
epoch: 1 step: 180 loss: 0.07153229415416718
epoch: 1 step: 190 loss: 0.06407849490642548
epoch: 1 step: 200 loss: 0.07519645243883133
epoch: 1 step: 210 loss: 0.08252124488353729
epoch: 1 step: 220 loss: 0.06278344243764877
epoch: 1 step: 230 l

In [ ]:
"""TEST"""
def get_char_count(arg1):
    c0 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c5 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*5:ALL_CHAR_SET_LEN*6])]
    c6 = ALL_CHAR_SET[np.argmax(arg1.cpu().tolist()[ALL_CHAR_SET_LEN*6:ALL_CHAR_SET_LEN*7])]
    return c0, c1, c2,c3, c4, c5, c6 
 


char_correct = 0
word_correct = 0
total = 0

betternet.eval()
lstm1.eval()

with torch.no_grad():
    for step, (img, label_oh, label) in enumerate(test_dl):
        char_count =0
        img = Variable(img).cuda()
        label_oh = Variable(label_oh.float()).cuda()
        #pred, feature = betternet(img)
        pred= betternet(img)

        #label_len = label[0]
        #pred = pred.squeeze(0)
        #label_oh = label_oh.squeeze(0)
        
        c0,c1,c2,c3,c4,c5,c6 = get_char_count(pred.squeeze()) 
        d0,d1,d2,d3,d4,d5,d6 = get_char_count(label_oh[0]) 
         
        c = '%s%s%s%s%s%s%s' % (c0, c1, c2, c3, c4, c5, c6)
        d = '%s%s%s%s%s%s%s' % (d0, d1, d2, d3, d4, d5, d6)
        #print(c)
        #print(d)
        #print(" ")
    
        char_count += (c0==d0)+(c1==d1)+(c2==d2)+(c3==d3)+(c4==d4)+(c5==d5)+(c6==d6)
        char_correct += char_count

        if(bool(str(label[0]) in str(c))):
            word_correct+=1

        total += 1
       
print(100/7*char_correct/total)
print(100*word_correct/total)
"""END TEST"""

KeyboardInterrupt: ignored

# 새 섹션

# 새 섹션